# Übung Datenbanken: Ein Zugauskunfssystem
![](https://www.klipfel-mathys.ch/images/imagesCAUM394L.jpg)

In [1]:
# Hier ist nur Code zum Initialisieren der Umgebeung, bitte gehen Sie weiter, es gibt nichts zu sehen.

# Keine langen Fehlermeldungen
import sys
ipython = get_ipython()

def exception_handler(exception_type, exception, traceback):
    print("%s: %s" % (exception_type.__name__, exception), file=sys.stderr)

ipython._showtraceback = exception_handler

# !wget -qOzugauskunft-kemper.db 'https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=zugauskunft-kemper.db'

# Lade die Erweiterung, damit wir SQL Befehle nutzen können
%reload_ext sql
%sql sqlite:///zugauskunft-kemper.db
%sql PRAGMA foreign_keys = ON

 * sqlite:///zugauskunft-kemper.db
Done.


[]

# Aufgabe

Nachdem wir in der letzten Aufgabe unsere Daten in folgendes Datenmodell importiert haben:

![](https://nextcloud.th-deg.de/s/Kc7m486Z3KbZyB4/download?path=%2F&files=bahn-er.png)

Wollen wir in dieser Aufgabe einige Abfragen auf diesem Modell entwerfen.

In [2]:
# !echo ".schema" | sqlite3 zugauskunft-kemper.db

In [3]:
# CREATE TABLE Zug (
#   ZugNr INTEGER NOT NULL,
#   Name TEXT NOT NULL,
#   StartBhf TEXT NOT NULL,
#   EndBhf TEXT NOT NULL,
#   FOREIGN KEY(StartBhf) REFERENCES Bahnhof(BahnhofId),
#   FOREIGN KEY(EndBhf) REFERENCES Bahnhof(BahnhofId),
#   PRIMARY KEY(ZugNr)
# );

# CREATE TABLE Bahnhof (
#   BahnhofId TEXT NOT NULL,
#   Name TEXT NOT NULL,
#   PRIMARY KEY(BahnhofId)
# );

# CREATE TABLE verbindet (
#   ZugNr Integer NOT NULL,
#   VonBahnhof TEXT NOT NULL,
#   NachBahnhof TEXT,
#   Abfahrt TIME NOT NULL,
#   Ankunft TIME NOT NULL,
#   FOREIGN KEY(VonBahnhof) REFERENCES Bahnhof(BahnhofId),
#   FOREIGN KEY(NachBahnhof) REFERENCES Bahnhof(BahnhofId),
#   FOREIGN KEY(ZugNr) REFERENCES Zug(ZugNr),
#   PRIMARY KEY(VonBahnhof, ZugNr)
# );

In [4]:
%%sql

SELECT * FROM Zug LIMIT 10

 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Name,StartBhf,EndBhf
1003122052,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122056,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122057,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122058,WBA1 Plattling,de:09276:5025_G,de:09271:4445
1003122059,WBA1 Plattling,de:09276:4726,de:09271:4445
1003122061,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122062,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122063,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122064,WBA1 Plattling,de:09271:5500,de:09271:4445
1003122065,WBA1 Plattling,de:09276:5025_G,de:09271:4445


In [5]:
%%sql

SELECT * FROM Bahnhof LIMIT 10

 * sqlite:///zugauskunft-kemper.db
Done.


BahnhofId,Name
de:09271:4445,Plattling
de:09271:5500,Deggendorf Hbf
de:09271:80502,Pankofen
de:09271:80587,Grafling-Arzting
de:09272:4688,Spiegelau
de:09272:4976_G,Klingenbrunn
de:09272:5047_G,Grafenau
de:09272:80738,Großarmschlag
de:09272:80739,Rosenau (b Grafenau)
de:09276:10969,Lichtenthal


In [6]:
%%sql

SELECT * FROM verbindet LIMIT 10

 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,VonBahnhof,NachBahnhof,Abfahrt,Ankunft
1003122052,de:09271:5500,de:09271:80502,05:30:00,05:30:00
1003122052,de:09271:80502,de:09271:4445,05:35:00,05:35:00
1003122052,de:09271:4445,None,05:39:00,05:39:00
1003122056,de:09271:5500,de:09271:80502,24:38:00,24:38:00
1003122056,de:09271:80502,de:09271:4445,24:44:00,24:45:00
1003122056,de:09271:4445,None,24:49:00,24:49:00
1003122057,de:09271:5500,de:09271:80502,23:29:00,23:29:00
1003122057,de:09271:80502,de:09271:4445,23:34:00,23:34:00
1003122057,de:09271:4445,None,23:39:00,23:39:00
1003122058,de:09276:5025_G,de:09276:80784,22:04:00,22:04:00


### Abfrage 1: Abfahrten in Deggendorf ✅

Entwerfen Sie eine Abfrage, die alle Abfahrten in Deggendorf anzeigt. Geben Sie die Zugnummer, den Namen, den Start- und Endbahnhof sowie die Abfahrtszeit aus.

In [7]:
%%sql

SELECT z.ZugNr, z.Name, b1.Name 'StartBhf', b2.Name 'EndBhf', v.Abfahrt
FROM Zug z
JOIN Bahnhof b1
ON(z.StartBhf = b1.BahnhofId)
JOIN Bahnhof b2
ON(z.EndBhf = b2.BahnhofId)
JOIN verbindet v
USING(ZugNr)
WHERE b1.Name LIKE '%Deggendorf%'
GROUP BY z.ZugNr
ORDER BY v.Abfahrt
LIMIT 10



 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Name,StartBhf,EndBhf,Abfahrt
1015320340,WBA1 Plattling,Deggendorf Hbf,Plattling,05:00:00
1003122052,WBA1 Plattling,Deggendorf Hbf,Plattling,05:30:00
1015320344,WBA1 Plattling,Deggendorf Hbf,Plattling,06:14:00
1003122120,WBA1 Plattling,Deggendorf Hbf,Plattling,06:45:00
1015320332,WBA1 Plattling,Deggendorf Hbf,Plattling,07:09:00
1003122088,WBA1 Plattling,Deggendorf Hbf,Plattling,07:38:00
1015320383,WBA1 Plattling,Deggendorf Hbf,Plattling,08:15:00
1015320334,WBA1 Plattling,Deggendorf Hbf,Plattling,08:45:00
1015320348,WBA1 Plattling,Deggendorf Hbf,Plattling,09:45:00
1015320361,WBA1 Plattling,Deggendorf Hbf,Plattling,10:45:00


Sie finden in den Ergebnissen sicher eine Zeilen mit : Abfahrt 25:27. Das ist typisch für Bahn Systeme, kann aber manche Zeitfunktion Ihrer Datenbank aus dem Tritt bringen.

## Aufgabe 2: Sortierng

Ihre Daten sind warscheinlich falsch sortiert, weil in den Daten '50000' angegeben ist. Der TIME  Datentyp in
SQLite versteht aber nur '050000'. Das können Sie dann über einen LIKE Match und einen UPDATE fixen.
Entwerfen Sie die entsprechene Query

## Abfrage 3: Laufweg einer zufälligen Fahrt ✅

Geben Sie alle Halte einer zufälligen (Hint: Subselect mit RANDOM()) Fahrt aus.

In [8]:
%%sql 

SELECT Zug.ZugNr, Zug.Name, verbindet.Ankunft, verbindet.Abfahrt, 
start.Name as VonBahnhof, end.Name as NachBahnhof
FROM Zug, verbindet, Bahnhof AS start, Bahnhof AS end 
WHERE  
verbindet.ZugNr = Zug.ZugNr AND  
verbindet.VonBahnhof = start.BahnhofId AND 
verbindet.NachBahnhof = end.BahnhofId AND
Zug.ZugNr = (SELECT ZugNr from Zug ORDER BY RANDOM() LIMIT 1)

 * sqlite:///zugauskunft-kemper.db
Done.


ZugNr,Name,Ankunft,Abfahrt,VonBahnhof,NachBahnhof
1003122289,"WBA4 Viechtach, Bahnhof/ZOB",04:50:00,04:50:00,Gotteszell,Ruhmannsfelden
1003122289,"WBA4 Viechtach, Bahnhof/ZOB",04:55:00,04:55:00,Ruhmannsfelden,"Patersdorf, Bahnhof"
1003122289,"WBA4 Viechtach, Bahnhof/ZOB",05:01:00,05:01:00,"Patersdorf, Bahnhof",Teisnach Rohde&Schwarz
1003122289,"WBA4 Viechtach, Bahnhof/ZOB",05:03:00,05:03:00,Teisnach Rohde&Schwarz,Teisnach
1003122289,"WBA4 Viechtach, Bahnhof/ZOB",05:15:00,05:08:00,Teisnach,Gumpenried
1003122289,"WBA4 Viechtach, Bahnhof/ZOB",05:22:00,05:22:00,Gumpenried,Schnitzmühle
1003122289,"WBA4 Viechtach, Bahnhof/ZOB",05:32:00,05:32:00,Schnitzmühle,"Viechtach, Bahnhof/ZOB"


## Abfrage 3: Von Deggendorf nach Grafenau

Wie komme ich von Deggendorf nach Grafenau? Gibt es einen einfachen Weg das in einer SQL Abfrage
abzubilden?